In [110]:
class yieldCurve:

    def __init__(self, model, sim_date, data):
        self.model = model
        self.sim_date = sim_date
        self.data = data

    def actual(self):
        return pd.DataFrame(self.data.loc[self.sim_date]).transpose()

    def simulate_curve(self):

        actual = self.actual().transpose()
        
        simulated_rates = {}
        for column in actual.index:
            if 'DGS' in column:
                initial_rate = self.data[column].loc[self.data.index < self.sim_date].iloc[-1]
                simulated_rates[column] = self.model.next_rate(initial_rate, 1/252)

        return pd.DataFrame(simulated_rates, index=[self.sim_date])

    def simulate_bond_prices(self, face_value):
        
        simulated_bond_prices = {}
        
        for column in self.data.columns:
            if 'DGS' in column:
                simulated_rate = self.simulate_curve()[column]/100  
                    
                maturity = (int(column.replace('DGS', '').replace('MO', '')) / 12) if 'MO' in column else int(column.replace('DGS', ''))
                # Converts nominal rate to continuous
                continuous_rate = np.log(1 + simulated_rate)
                    
                bond_price = self.model.bond_price(continuous_rate, maturity, 0) * face_value
                simulated_bond_prices[column] = bond_price

        return pd.DataFrame(simulated_bond_prices)

    def actual_bond_prices(self, face_value):

        actual = self.actual()        
        actual_bond_prices = {}
        
        for column in self.data.columns:
            if 'DGS' in column:
                maturity = int(column.replace('DGS', '').replace('MO', '')) / 12 if 'MO' in column else int(column.replace('DGS', ''))
                actual_rate = actual[column]/100
                bond_price = self.model.bond_price(actual_rate, maturity, 0) * face_value
                actual_bond_prices[column] = bond_price
        return pd.DataFrame(actual_bond_prices)

    def comb_data(self):

        actual_rate = self.actual()
        sim_rate = self.simulate_curve()
        comb_rates =  pd.DataFrame({'Actual Rates': actual_rate.iloc[0],
                                        'Simulated Rates': sim_rate.iloc[0]})
    
        actual_price = self.actual_bond_prices(1000)
        sim_price = self.simulate_bond_prices(1000)
    
        comb_prices =  pd.DataFrame({'Actual Prices': actual_price.iloc[0],
                                         'Simulated Prices': sim_price.iloc[0]})
            
        return comb_rates, comb_prices

In [101]:
import pandas as pd
from ECIRModel import ECIRModel
from CIRModel import BasicCIRModel
import numpy as np

In [102]:
# Load the DGS_30 data
dgs_30_data = pd.read_csv('Data Folder/DGS_30.csv')
# Load the DGS3MO data
dgs3mo_data = pd.read_csv('Data Folder/DGS3MO.csv')

# Convert the 'DATE' column to datetime and set it as index for both DataFrames
dgs_30_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
dgs_30_data['Date'] = pd.to_datetime(dgs_30_data['Date'])
dgs_30_data.set_index('Date', inplace=True)

dgs3mo_data.rename(columns={'DATE': 'Date'}, inplace=True)
dgs3mo_data['Date'] = pd.to_datetime(dgs3mo_data['Date'])
dgs3mo_data.set_index('Date', inplace=True)

# Insert the DGS3MO data into the DGS_30 data as the first column
dgs_30_data.insert(0, 'DGS3MO', dgs3mo_data['DGS3MO'])

data=dgs_30_data
# Check the result
data.head()

,DGS3MO,DGS1,DGS2,DGS3,DGS4,DGS5,DGS6,DGS7,DGS8,DGS9,...,DGS21,DGS22,DGS23,DGS24,DGS25,DGS26,DGS27,DGS28,DGS29,DGS30
Date,,,,,,,,,,,,,,,,,,,,,
1981-09-01,17.010,17.0600,16.780,16.57,16.350396,16.120,15.898415,15.710,15.573295,15.478799,...,15.070462,15.049236,15.025332,14.997763,14.965539,14.927672,14.883173,14.831054,14.770326,14.7000
1981-09-02,16.650,17.1600,16.780,16.43,16.225968,16.090,15.924878,15.750,15.604572,15.490415,...,15.103136,15.093330,15.078842,15.057928,15.028848,14.989861,14.939224,14.875196,14.796035,14.7000
1981-09-03,16.960,17.3100,16.900,16.48,16.262789,16.140,15.967593,15.780,15.640102,15.546094,...,15.194779,15.177243,15.156281,15.130776,15.099616,15.061687,15.015875,14.961065,14.896145,14.8200
1981-09-04,16.640,17.2400,16.890,16.52,16.302912,16.170,16.019604,15.860,15.720523,15.604793,...,15.237907,15.232570,15.222048,15.204398,15.177676,15.139941,15.089248,15.023655,14.941220,14.8400
1981-09-07,16.565,17.2775,16.935,16.55,16.336354,16.215,16.066772,15.905,15.767511,15.657616,...,15.328694,15.324179,15.314622,15.298188,15.273045,15.237358,15.189294,15.127019,15.048698,14.9525


In [103]:
dates = data.loc[data.index >= '2024-03-19'].index

params_ecir = {
    "kappa": 0.01227,    
    "mu_r": 0.09854,     
    "sigma": 0.09395,    
    "mu": 0.002995,      
    "gamma": 0.002321,   
    "r": 66,             
    "p": 0.3049          
}

params_cir = {
    "kappa": 1.2310,
    "mu_r": 0.0459,
    "sigma": 0.1410
}

In [111]:
test = yieldCurve(ECIRModel(**params_ecir), dates[0], data)
test_rate, test_price = test.comb_data()

In [112]:
test_price

,Actual Prices,Simulated Prices
DGS3MO,986.750760,986.727059
DGS1,951.837045,951.726819
DGS2,912.583416,910.965370
DGS3,877.051743,874.131095
DGS4,843.144322,839.944914
DGS5,809.786016,807.309553
DGS6,776.642152,775.157266
DGS7,744.249210,743.590907
DGS8,713.506684,715.183897
DGS9,684.368064,691.105690


In [113]:
test2 = yieldCurve(BasicCIRModel(**params_cir), dates[0], data)
test_rate, test_price = test2.comb_data()

In [114]:
test_price

,Actual Prices,Simulated Prices
DGS3MO,986.750760,987.057379
DGS1,951.837045,953.432287
DGS2,912.583416,912.486377
DGS3,877.051743,873.132864
DGS4,843.144322,834.938807
DGS5,809.786016,798.031097
DGS6,776.642152,762.526894
DGS7,744.249210,728.522660
DGS8,713.506684,696.062967
DGS9,684.368064,665.057939
